In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import kaggle

In [12]:
def scrape_movie_urls(base_url, num_movies):
    movie_urls = []
    page = 1
    
    while len(movie_urls) < num_movies:
        response = requests.get(f"{base_url}page/{page}/")
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all movie link elements (inspect the website to get the correct class or tag)
        movie_elements = soup.find_all('a', class_='film-poster')
        
        for element in movie_elements:
            if len(movie_urls) >= num_movies:
                break
            
            movie_url = 'https://letterboxd.com' + element.get('href')
            movie_urls.append(movie_url)
        
        page += 1
    
    return movie_urls

# Example usage: scrape 100 movie URLs
movie_urls = scrape_movie_urls("https://letterboxd.com/films/popular/", 100)
print(movie_urls)

KeyboardInterrupt: 

In [22]:
base_url="https://letterboxd.com/films/popular/"
page=1
response = requests.get(f"{base_url}page/{page}/")
soup = BeautifulSoup(response.content, 'html.parser')


[]

In [23]:
movie_elements = soup.find_all('a', class_='frame has-menu')
movie_elements

[]

In [25]:
soup.find_all('a', href=lambda x: x and '/film/' in x)

[]

In [26]:
f"{base_url}page/{page}/"

'https://letterboxd.com/films/popular/page/1/'

In [ ]:
<a href="/film/parasite-2019/" class="frame has-menu" data-original-title="Parasite (2019) 4.56"><span class="frame-title">Parasite (2019)</span><span class="overlay"></span><span class="overlay-actions js-film-options -w70" style="display: none;"><span class="film-watch-link-target" data-film-id="426406"><span class="film-watch-link"><span class="has-icon icon-16 icon-watch ajax-click-action" data-action="/film/parasite-2019/mark-as-watched/"><span class="replace icon"></span>Seen this film?</span></span></span><span class="like-link-target" data-likeable-uid="film:426406"><span class="like-link"><span class="has-icon icon-16 ajax-click-action  icon-like" data-action="/s/film:426406/like/" data-recaptcha-action="film_like" data-confirm="" data-modal-confirm="true"><span class="icon"></span>Like this film?</span></span></span><span class="replace menu-link icon" style="opacity: 1;"></span></span></a>

In [31]:
soup.prettify()

65047

In [11]:
rating_map = {
    '★★★★★': 1.0,
    '★★★★½': 0.9,
    '★★★★': 0.8,
    '★★★½': 0.7,
    '★★★': 0.6,
    '★★½': 0.5,
    '★★': 0.4,
    '★½': 0.3,
    '★': 0.2,
    '½': 0.1,
}

def normalize_rating(star_rating):
    return rating_map.get(star_rating, None)

In [41]:
def scrape_letterboxd_reviews(movie_url, num_reviews):
    reviews = []
    page = 1
    while len(reviews) < num_reviews:        
        response = requests.get(f"{movie_url}/reviews/page/{page}/")
        soup = BeautifulSoup(response.content, 'html.parser')
        review_elements = soup.find_all('div', class_='film-detail-content')
        
        for element in review_elements:
            if len(reviews) >= num_reviews:
                break
            
            review_text_element = element.find('div', class_='body-text -prose collapsible-text')
            review_text = review_text_element.get_text(strip=True) if review_text_element else None
            
            rating_element = element.find('span', class_='rating')
            star_rating = rating_element.get_text(strip=True) if rating_element else None
            
            if review_text and star_rating is not None:
                reviews.append({'review': review_text, 'rating': normalize_rating(star_rating) })
        
        page += 1

    return pd.DataFrame(reviews)

In [42]:
# Example usage
movie_url = 'https://letterboxd.com/film/deadpool-wolverine'  # Replace with actual movie URL
num_reviews = 300
reviews_df = scrape_letterboxd_reviews(movie_url, num_reviews)
#reviews_df.to_csv('data/reviews.csv', index=False)
reviews_df.head()

,review,rating
0,If it's not too meta is it even a Deadpool fil...,0.5
1,Genuinely good fight scenes I know it’s Stefan...,0.7
2,Ooh I been waiting a long time fo dis,1.0
3,Very funny. The plot was hard to follow but so...,0.8
4,who else replayed the end cus of wolverine,0.8


In [43]:
np.sort( reviews_df.rating.unique()) # to double check that the rating hashmap working and non nan

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

jovyan
